In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import re, os, csv, pathlib, ast
import pandas as pd
from statistics import mean, variance

accent_short_forms = {"hindi":"HIN", "korean":"KOR", "vietnamese":"VTN", "arabic":"ARB", "chinese":"CHN", "spanish":"ESP"}
accent_map = {"ABA":"arabic","SKA":"arabic","YBAA":"arabic","ZHAA":"arabic",
              "BWC":"chinese","LXC":"chinese","NCC":"chinese","TXHC":"chinese",
              "ASI":"hindi","RRBI":"hindi","SVBI":"hindi","TNI":"hindi",
              "HJK":"korean","HKK":"korean","YDCK":"korean","YKWK":"korean",
              "EBVS":"spanish","ERMS":"spanish","MBMPS":"spanish","NJS":"spanish",
              "HQTV":"vietnamese","PNV":"vietnamese","THV":"vietnamese","TLV":"vietnamese"
              }
raw_string="""|ABA|M|Arabic|1129|150|\n|SKA|F|Arabic|974|150|\n|YBAA|M|Arabic|1130|149||ZHAA|F|Arabic|1132|150|\n|BWC|M|Chinese|1130|150|\n|LXC|F|Chinese|1131|150|\n|NCC|F|Chinese|1131|150|\n|TXHC|M|Chinese|1132|150|\n|ASI|M|Hindi|1131|150|\n|RRBI|M|Hindi|1130|150|\n|SVBI|F|Hindi|1132|150|\n|TNI|F|Hindi|1131|150|\n|HJK|F|Korean|1131|150|\n|HKK|M|Korean|1131|150|\n|YDCK|F|Korean|1131|150|\n|YKWK|M|Korean|1131|150|\n|EBVS|M|Spanish|1007|150|\n|ERMS|M|Spanish|1132|150|\n|MBMPS|F|Spanish|1132|150|\n|NJS|F|Spanish|1131|150|\n|HQTV|M|Vietnamese|1132|150|\n|PNV|F|Vietnamese|1132|150|\n|THV|F|Vietnamese|1132|150|\n|TLV|M|Vietnamese|1132|150|"""
raw_strings=raw_string.split('\n')
gender_map={}
for lne in raw_strings:
    attrs=lne.split('|')
    gender_map[attrs[1]]=attrs[2]

composed_accent_map = {k: accent_short_forms.get(v) for k, v in accent_map.items()}

def replace_with_short_forms(s):
    for key, value in accent_short_forms.items():
        s = s.replace(key, value)
    return s

def group_speakers(s):
    ret = {}
    speaker_counts = ast.literal_eval(s)
    for speaker, count in speaker_counts.items():
        accent = composed_accent_map[speaker]
        if accent not in ret:
            ret[accent] = {}
        ret[accent][speaker] = count
    return str(ret)

def group_accents(s):
    ret = {}
    speaker_counts = ast.literal_eval(s)
    for speaker, count in speaker_counts.items():
        accent = composed_accent_map[speaker]
        if accent not in ret:
            ret[accent] = {}
        ret[accent][speaker] = count
    accent_counts={}
    for accent in ret:
        cnt=0
        for speaker in ret[accent]:
            cnt+=ret[accent][speaker]
        accent_counts[accent]=cnt
    return str(accent_counts)

def last_name(pth):
    return pathlib.PurePath(pth).name

def get_dirs(pth):
    return [last_name(f.name) for f in os.scandir(pth) if f.is_dir()]

def get_each_run(lne):
    return list(map(float, re.findall(': (.+?) -> ', lne)[0].split(' ')))

def get_selection_counts(s):
    return list(map(group_accents, re.findall('Counter\\((.+?)\\)', s)))+list(map(group_speakers, re.findall('Counter\\((.+?)\\)', s)))

def get_test_file_from_stats_path(run_number, stats_file_opened):
    return stats_file_opened.name[:-9]+"run_{}/output/test_infer_log.txt".format(run_number)

def WER_test_file(test_file):
    txt_file = open(test_file, 'r')
    lines = txt_file.readlines()
    matched = ""
    for line in lines:
        if "==========>>>>>>Evaluation Greedy WER: " in line:
            txt_file.close()
            return float(line.rstrip().split(": ")[1])
    txt_file.close()
    return ""

def get_eta(func, eta):
    return "-n:"+str(float(eta[4:]))

In [3]:
budget = 100
target = 50
features = '39'

In [5]:
# sample_path = 'Error-Driven-ASR-Personalization/CMU_expts/accent/hindi/manifests/TSS_output/all/budget_100/target_50/FL1MI/eta_1.0/euclidean/39/stats.txt'
cols = ['speaker', 'ground', 'function', 'similarity', 'duration', 'samples', 
        'WER-r1', 'WER-r2', 'WER-r3', 'WER-mean', 'WER-var', 'accents_run1', 'accents_run2', 'accents_run3', 'speakers_run1', 'speakers_run2', 'speakers_run3']
df = pd.DataFrame(columns = cols)

# speakers = [f.name for f in os.scandir('./') if f.is_dir() and f.name != '.ipynb_checkpoints']
speakers = ['ABA', 'ASI', 'BWC', 'EBVS', 'HJK', 'ERMS', 'HKK', 'HQTV']

cnt = 0
for speaker in speakers:
    if not(pathlib.Path('./{}/manifests/TSS_output/'.format(speaker)).is_dir()):
        continue
    for pick_from in get_dirs('./{}/manifests/TSS_output/'.format(speaker)):
        if not(pathlib.Path('./{}/manifests/TSS_output/{}/budget_{}/target_{}/'.format(speaker, pick_from, budget, target)).is_dir()):
            continue
        for function in get_dirs('./{}/manifests/TSS_output/{}/budget_{}/target_{}/'.format(speaker, pick_from, budget, target)):
            if function == "random":
                stats_file_path = './{}/manifests/TSS_output/{}/budget_{}/target_{}/{}/stats.txt'.format(speaker, pick_from, budget, 
                                                                                                                target, function)                                                                                  
                if not(os.path.isfile(stats_file_path)):
                    continue
                stats_file = open(stats_file_path, 'r')
                lines = stats_file.readlines()
                total_selections, total_durations, speakered_selections, speakered_durations = map(get_each_run, lines[:4])
                sample_frac = mean([x[0]/x[1] for x in zip(speakered_selections, total_selections)])
                sample_total = mean(total_selections)
                duration_frac = mean([x[0]/x[1] for x in zip(speakered_durations, total_durations)])
                duration_total = mean(total_durations)
                df_duration = "{:.2f}/{:.2f}".format(duration_total*duration_frac, duration_total)
                df_samples = "{:.2f}/{:.2f}".format(sample_total*sample_frac, sample_total)
                df_selections = get_selection_counts(lines[5])
                wers = [WER_test_file(get_test_file_from_stats_path(i, stats_file)) for i in range(1,4)]
                df_wer_mean = round(mean(wers), 2)
                df_wer_var = round(variance(wers), 4)
                speaker_new=speaker+"[{}/{}]".format(composed_accent_map[speaker], gender_map[speaker])
                df = df.append(dict(zip(cols, [speaker_new, pick_from, function, "NA", df_duration, df_samples]+wers
                                                +[df_wer_mean, df_wer_var] + df_selections)), ignore_index=True)
                stats_file.close()
                continue
            for eta in get_dirs('./{}/manifests/TSS_output/{}/budget_{}/target_{}/{}/'.format(speaker, pick_from, budget, target, function)):
                for similarity in get_dirs('./{}/manifests/TSS_output/{}/budget_{}/target_{}/{}/{}/'.format(speaker, pick_from, 
                                                                                                        budget, target, function, eta)):
                    # print(cnt)
                    cnt += 1
                    stats_file_path = './{}/manifests/TSS_output/{}/budget_{}/target_{}/{}/{}/{}/{}/stats.txt'.format(speaker, pick_from, budget, 
                                                                                                                target, function, eta, similarity, 
                                                                                                                features)                                                                                  
                    if not(os.path.isfile(stats_file_path)):
                        continue
                    stats_file = open(stats_file_path, 'r')
                    lines = stats_file.readlines()
                    total_selections, total_durations, speakered_selections, speakered_durations = map(get_each_run, lines[:4])
                    sample_frac = mean([x[0]/x[1] for x in zip(speakered_selections, total_selections)])
                    sample_total = mean(total_selections)
                    duration_frac = mean([x[0]/x[1] for x in zip(speakered_durations, total_durations)])
                    duration_total = mean(total_durations)
                    df_duration = "{:.2f}/{:.2f}".format(duration_total*duration_frac, duration_total)
                    df_samples = "{:.2f}/{:.2f}".format(sample_total*sample_frac, sample_total)
                    df_selections = get_selection_counts(lines[5])
                    wers = [WER_test_file(get_test_file_from_stats_path(i, stats_file)) for i in range(1,4)]
                    df_wer_mean = round(mean(wers), 2)
                    df_wer_var = round(variance(wers), 4)
                    speaker_new=speaker+"[{}/{}]".format(composed_accent_map[speaker], gender_map[speaker])
                    df = df.append(dict(zip(cols, [speaker_new, pick_from, function+get_eta(function, eta), similarity, df_duration, df_samples]+
                                                   wers+[df_wer_mean, df_wer_var] + df_selections)), 
                                   ignore_index=True)
                    stats_file.close()

In [7]:
df = df.sort_values(by=['speaker', 'similarity', 'ground', 'function'], ascending=True, ignore_index=True)
display(df)

,speaker,ground,function,similarity,duration,samples,WER-r1,WER-r2,WER-r3,WER-mean,WER-var,accents_run1,accents_run2,accents_run3,speakers_run1,speakers_run2,speakers_run3


In [8]:
# total selection : 100 100 100 -> 100.00
# total selection duration: 357.0149433106577 357.0149433106577 357.0149433106577 -> 357.01
# accented selection: 76 76 76 -> 76.00
# accented duration: 254.74947845804974 254.74947845804974 254.74947845804974 -> 254.75

# all selections: [Counter({'hindi': 76, 'korean': 8, 'spanish': 7, 'arabic': 3, 'chinese': 3, 'vietnamese': 3}), Counter({'hindi': 76, 'korean': 8, 'spanish': 7, 'arabic': 3, 'chinese': 3, 'vietnamese': 3}), Counter({'hindi': 76, 'korean': 8, 'spanish': 7, 'arabic': 3, 'chinese': 3, 'vietnamese': 3})]

#Evaluation Greedy WER: 16.19

df.to_csv("report_{}_{}_{}.csv".format(budget, target, features), index=False)